In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.spatial import procrustes
data_path = '/Users/jamesburgess/research/robust-dim-reduction-popgen'

In [6]:
popres_samples = pd.read_csv('{}/data/labels.csv'.format(data_path))
popres_samples.groupby('label').count()

,Unnamed: 0,indID
label,,
Albania,6,6
Austria,28,28
Belgium,40,40
Bosnia-Herzegovina,18,18
Bulgaria,4,4
Croatia,16,16
Cyprus,8,8
Czech-Republic,22,22
Denmark,2,2
